# Quick analysis

In [1]:
## Import libraries

%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from IPython.display import display, Markdown


In [2]:
# import data

#df = pd.read_csv("../output/input.csv")
df = pd.read_feather("../output/input.feather")

maxdate = df["hospital_admission_date"].fillna("1900-01-01").max()
print(maxdate)

2022-03-22


In [3]:
df1 = df.loc[df["outpatient_covid_therapeutic_date"] <= maxdate]
summary1 = pd.DataFrame(df1[["outpatient_covid_therapeutic_date", "sus_treatment_date",
                        "hospital_admission_date","hospital_admission_x892_date","hospital_admission_x292_date",
                        "hospital_attendance_date"]].count()).rename(columns={0:"count"})
summary1["percent"] = (100*(summary1["count"]/summary1["count"]["outpatient_covid_therapeutic_date"])).round(2).astype(int)
display(summary1)

,count,percent
outpatient_covid_therapeutic_date,801,100
sus_treatment_date,82,10
hospital_admission_date,43,5
hospital_admission_x892_date,36,4
hospital_admission_x292_date,41,5
hospital_attendance_date,42,5


In [4]:
for col in ["region_covid_therapeutics","age_group"]:
    summary2 = pd.DataFrame(df1.groupby(col)\
               [["outpatient_covid_therapeutic_date","sus_treatment_date"]].count()).rename(columns={0:"count"})
    summary2["percent"] = (100*summary2["sus_treatment_date"]/summary2["outpatient_covid_therapeutic_date"]).round(2).astype(int)
    display(summary2)

,outpatient_covid_therapeutic_date,sus_treatment_date,percent
region_covid_therapeutics,,,
East,70,10,14
East Midlands,82,6,7
London,173,20,11
North East,82,8,9
North West,71,9,12
South East,73,4,5
South West,97,13,13
West Midlands,95,8,8
Yorkshire and The Humber,58,4,6


,outpatient_covid_therapeutic_date,sus_treatment_date,percent
age_group,,,
12-24,75,3,4
25-34,82,9,10
35-44,172,23,13
45-54,165,21,12
55-64,74,5,6
65-74,77,10,12
75+,76,6,7
missing,80,5,6
